In [3]:
ls

2_to_0.py                     generator.py
300_unetAdam.6683.hal01.err   log.out
300_unetAdam.6683.hal01.out   main.py
400_unetAdam.6663.hal01.err   model.py
400_unetAdam.6663.hal01.out   output/
500_Adam.6218.hal03.out       prediction.py
500_unetAdam.6633.hal06.err   __pycache__/
500_unetAdam.6633.hal06.out   reload_and_predict.ipynb
500_unetAdam.6657.hal01.err   remake_data.py
500_unetAdam.6657.hal01.out   rename.py
500_VGG16Adam.6221.hal03.err  run_script.sb
500_VGG16Adam.6221.hal03.out  train.py
50_Adam.6217.hal03.out        tryout.ipynb
aug_main.py                   utils.py
custom_generator.py           vis.ipynb
custom_generator.pyc


In [4]:
import numpy as np
image = np.load('npy/train_frames/mclean_roi_mb_119_080.npy')

FileNotFoundError: [Errno 2] No such file or directory: 'npy/train_frames/mclean_roi_mb_119_080.npy'

In [18]:
a = np.arange(48)
a = a.reshape((2,3,4,2))
print(a)

[[[[ 0  1]
   [ 2  3]
   [ 4  5]
   [ 6  7]]

  [[ 8  9]
   [10 11]
   [12 13]
   [14 15]]

  [[16 17]
   [18 19]
   [20 21]
   [22 23]]]


 [[[24 25]
   [26 27]
   [28 29]
   [30 31]]

  [[32 33]
   [34 35]
   [36 37]
   [38 39]]

  [[40 41]
   [42 43]
   [44 45]
   [46 47]]]]


In [19]:
b = np.arange(48).reshape((2,3,4,2))*2
print(b)

[[[[ 0  2]
   [ 4  6]
   [ 8 10]
   [12 14]]

  [[16 18]
   [20 22]
   [24 26]
   [28 30]]

  [[32 34]
   [36 38]
   [40 42]
   [44 46]]]


 [[[48 50]
   [52 54]
   [56 58]
   [60 62]]

  [[64 66]
   [68 70]
   [72 74]
   [76 78]]

  [[80 82]
   [84 86]
   [88 90]
   [92 94]]]]


In [21]:
tuple(range(1, len(a.shape)-1)) 

(1, 2)

In [35]:
tuple(range(1, 3))

(1, 2)

In [29]:
tuple(range(1, 3))

(1, 2)

In [36]:
numerator = 2. * np.sum(a * b, tuple(range(1, len(a.shape)-1)) )
print(numerator)

[[ 8096.  9200.]
 [61088. 64496.]]


In [28]:
print(image[:,:,0])

[[2.996763   2.2393272  1.7381352  ... 1.5808836  1.1994978  0.79860324]
 [2.4011664  1.9628491  1.8026408  ... 2.0577664  1.7570992  1.2959198 ]
 [2.886951   3.9729075  3.99233    ... 1.0628033  0.8858251  0.9703405 ]
 ...
 [1.624238   1.3135468  1.3770803  ... 1.1922654  1.1949275  1.162187  ]
 [1.7010466  1.7360045  1.9306469  ... 1.3608227  1.0138186  0.73196596]
 [2.440472   2.2169726  2.3815024  ... 0.98271006 0.6483698  0.6239562 ]]


In [24]:
from PIL import Image
new = Image.fromarray(image[:,:,0]).convert('L')
img = new.resize((256,256))

In [25]:
print(np.array(img))

[[2 2 1 ... 1 1 0]
 [2 1 1 ... 2 1 1]
 [2 3 3 ... 1 0 0]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 0]
 [2 2 2 ... 0 0 0]]


In [3]:
image.dtype

dtype('float32')

In [2]:
np.max(image)

2.0

In [4]:
np.min(image)

1.0

In [5]:
BATCH_SIZE = 16
NO_OF_EPOCHS = 10

In [4]:
from custom_generator import *
from generator import *
from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping
import os
import tensorflow as tf
import keras.backend as K
import model
from model import dice_coef_loss, dice_coef
from utils import *
import numpy as np
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv2D, Reshape
from keras.layers.core import Dropout, Activation
from keras.models import Model
from keras.optimizers import Adadelta, Adam
import matplotlib.pyplot as plt
import time
from functools import *

BATCH_SIZE = 32
NO_OF_EPOCHS = 5
Model_name = '500_UnetAdadelta_aug_5ep'
result_name = '500_UnetAdadelta_aug_5ep'

Checkpoint_path = '/home/yifanc3/checkpoints/%s/' % Model_name
if not os.path.isdir(Checkpoint_path):
    os.makedirs(Checkpoint_path)    
    
#TRAIN
train_frame_path = '/home/yifanc3/dataset/npy/no_shuffle/train_frames'
train_mask_path = '/home/yifanc3/dataset/npy/no_shuffle/train_masks'

val_frame_path = '/home/yifanc3/dataset/npy/no_shuffle/val_frames'
val_mask_path = '/home/yifanc3/dataset/npy/no_shuffle/val_masks'

# train_gen = data_gen(train_frame_path,train_mask_path, batch_size = BATCH_SIZE)
# val_gen = data_gen(val_frame_path,val_mask_path, batch_size = BATCH_SIZE)
train_gen = train_gen_aug(train_frame_path, train_mask_path, BATCH_SIZE)
val_gen = val_gen_aug(val_frame_path, val_mask_path, BATCH_SIZE)

# Train the model
NO_OF_TRAINING_IMAGES = len(os.listdir('/home/yifanc3/dataset/npy/no_shuffle/train_frames/'))
NO_OF_VAL_IMAGES = len(os.listdir('/home/yifanc3/dataset/npy/no_shuffle/val_frames/'))

#NO_OF_EPOCHS = 'ANYTHING FROM 30-100 FOR SMALL-MEDIUM SIZED DATASETS IS OKAY'

#BATCH_SIZE = 'BATCH SIZE PREVIOUSLY INITIALISED'

weights_path = '/home/yifanc3/checkpoints/%s/weights.{epoch:02d}-{val_loss:.2f}.hdf5' % Model_name

#vgg16_32s
inputs = Input((256,256,5))
base = model.get_unet(inputs)
# print('base shape,',base.get_shape())
m = Model(inputs=inputs, outputs=base)
m.summary()

opt = Adam(lr=1E-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

# m.compile(loss=soft_dice_loss,
#                optimizer=opt,
#                metrics=[Mean_IOU])


start = time.time()

ncce = partial(w_categorical_crossentropy)

m.compile( optimizer = Adadelta(), loss = pixel_wise_loss, metrics = [Mean_IOU,'accuracy'])

checkpoint = ModelCheckpoint(weights_path, monitor='val_loss',
                             verbose=1, save_best_only=True, mode='min')

csv_logger = CSVLogger('./log.out', append=True, separator=';')

earlystopping = EarlyStopping(monitor = 'val_loss', verbose = 1,
                             min_delta = 0.01, patience = 3, mode = 'min')

callbacks_list = [checkpoint, csv_logger] #, earlystopping]

history = m.fit_generator(train_gen, epochs=NO_OF_EPOCHS,
                          steps_per_epoch = (NO_OF_TRAINING_IMAGES//BATCH_SIZE),
                          validation_data=val_gen,
                          validation_steps=(NO_OF_VAL_IMAGES//BATCH_SIZE),
                          callbacks=callbacks_list)


Model_path = '/home/yifanc3/models/%s/' % Model_name
if not os.path.isdir(Model_path):
    os.makedirs(Model_path)
    
    
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['Mean_IOU'])
plt.plot(history.history['val_Mean_IOU'])
plt.title('model Mean_IOU')
plt.ylabel('Mean_IOU')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(os.path.join(Model_path,'Mean_IoU.png'))
plt.clf()
plt.cla()
plt.close()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig(os.path.join(Model_path,'loss.png'))


Model_path = '/home/yifanc3/models/%s/' % Model_name
if not os.path.isdir(Model_path):
    os.makedirs(Model_path)

m.save(os.path.join(Model_path,'%s.h5' %Model_name))
end = time.time()
print('train_time = %d' %(end - start))

#TEST
print('======Start Testing======')
   
test_frame_path = '/home/yifanc3/dataset/npy/no_shuffle/test_frames'
test_mask_path = '/home/yifanc3/dataset/npy/no_shuffle/test_masks'


# test_gene = test_gen(test_frame_path, test_mask_path)
# results = m.predict_generator(test_gene, 30, verbose=1)

X,Y = test_gen(test_frame_path, test_mask_path)

score = m.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (m.metrics_names[1], score[1]*100))
# print("%s: %.2f%%" % (m.metrics_names[2], score[2]*100))
# print("%s: %.2f%%" % (m.metrics_names[3], score[3]*100))
# print("%s: %.2f%%" % (m.metrics_names[4], score[4]*100))
# print("%s: %.2f%%" % (m.metrics_names[5], score[5]*100))
# print("%s: %.2f%%" % (m.metrics_names[6], score[6]*100))

results = m.predict(X)
new_r = np.argmax(results,axis=-1)

#save image
save_frame_path = "/home/yifanc3/results/orig_frames_256"
result_path = os.path.join("/home/yifanc3/results/",result_name)

if not os.path.isdir(result_path):
    os.makedirs(result_path)
    
if not os.path.isdir(save_frame_path):
    os.makedirs(save_frame_path)
    
saveResult(result_path, test_mask_path,results)
# saveFrame_256(save_frame_path, test_frame_path, X)






NameError: name 'datagen' is not defined

In [7]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

import model

NO_OF_TRAINING_IMAGES = len(os.listdir('/home/yifanc3/npy/train_frames/'))
NO_OF_VAL_IMAGES = len(os.listdir('/home/yifanc3/npy/val_frames/'))

#NO_OF_EPOCHS = 'ANYTHING FROM 30-100 FOR SMALL-MEDIUM SIZED DATASETS IS OKAY'

#BATCH_SIZE = 'BATCH SIZE PREVIOUSLY INITIALISED'

weights_path = '/home/yifanc3/checkpoints/v1_weight/'

m = model.FCN_Vgg16_32s()
opt = Adam(lr=1E-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

m.compile(loss='dice_loss',
              optimizer=opt,
              metrics= mIOU)

checkpoint = ModelCheckpoint(weights_path, monitor='METRIC_TO_MONITOR', 
                             verbose=1, save_best_only=True, mode='max')

csv_logger = CSVLogger('./log.out', append=True, separator=';')

earlystopping = EarlyStopping(monitor = 'METRIC_TO_MONITOR', verbose = 1,
                              min_delta = 0.01, patience = 3, mode = 'max')

callbacks_list = [checkpoint, csv_logger, earlystopping]

results = m.fit_generator(train_gen, epochs=NO_OF_EPOCHS, 
                          steps_per_epoch = (NO_OF_TRAINING_IMAGES//BATCH_SIZE),
                          validation_data=val_gen, 
                          validation_steps=(NO_OF_VAL_IMAGES//BATCH_SIZE), 
                          callbacks=callbacks_list)
m.save('Model.h5')

ImportError: No module named 'keras'

In [27]:
import os
test_mask_dir = '/home/yifanc3/dataset/tif/val_masks/val_masks/'

In [28]:
masks_files = os.listdir(test_mask_dir)

In [29]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from libtiff import TIFF

In [30]:
file = os.path.join(test_mask_dir + masks_files[0])


In [31]:
tif = TIFF.open(file)
img = tif.read_image()
# img = np.load(MASK_PATH+image)
#img.save(DATA_PATH+'/{}'.format(dir_name)+'/'+image)
# filename = '/home/yifanc3/dataset/tif/'+'{}'.format(dir_name)+'/'+image[-11:-3]+'tif'
# print(filename)
# tif = TIFF.open(filename, mode='w')
# tif.write_image(img)

In [32]:
np.unique(img)

array([1., 2.], dtype=float32)